In [58]:
	
import os
import dotenv
from pathlib import Path
from sentence_transformers import SentenceTransformer
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders.text import TextLoader
from langchain_community.document_loaders import (
    WebBaseLoader, 
    PyPDFLoader, 
    Docx2txtLoader,
)
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

dotenv.load_dotenv()


True

In [59]:
# Load docs

doc_paths = [
    "docs/test_rag.pdf",
    "docs/test_rag.docx",
]

docs = [] 
for doc_file in doc_paths:
    file_path = Path(doc_file)

    try:
        if doc_file.endswith(".pdf"):
            loader = PyPDFLoader(file_path)
        elif doc_file.endswith(".docx"):
            loader = Docx2txtLoader(file_path)
        elif doc_file.endswith(".txt") or doc_file.name.endswith(".md"):
            loader = TextLoader(file_path)
        else:
            print(f"Document type {doc_file.type} not supported.")
            continue

        docs.extend(loader.load())

    except Exception as e:
        print(f"Error loading document {doc_file.name}: {e}")


# Load URLs

url = "https://docs.streamlit.io/develop/quick-reference/release-notes"
try:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())

except Exception as e:
    print(f"Error loading document from {url}: {e}")

In [60]:
docs

[Document(metadata={'source': 'docs/test_rag.pdf', 'page': 0}, page_content='My favorite food is margarita pizza. \nThere are 47588 bottles in the truck. '),
 Document(metadata={'source': 'docs/test_rag.docx'}, page_content='My favorite food is margarita pizza.\n\nThere are 47588 bottles in the truck.'),
 Document(metadata={'source': 'https://docs.streamlit.io/develop/quick-reference/release-notes', 'title': 'Release notes - Streamlit Docs', 'description': 'A changelog of highlights and fixes for each version of Streamlit.', 'language': 'No language found.'}, page_content='Release notes - Streamlit DocsDocumentationsearchSearchrocket_launchGet startedInstallationaddFundamentalsaddFirst stepsaddcodeDevelopConceptsaddAPI referenceaddTutorialsaddQuick referenceremoveCheat sheetRelease notesremove202420232022202120202019Pre-release featuresRoadmapopen_in_newweb_assetDeployConceptsaddStreamlit Community CloudaddSnowflakeOther platformsaddschoolKnowledge baseFAQInstalling dependenciesDeploym

In [61]:
# Split docs

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,
    chunk_overlap=1000,
)

document_chunks = text_splitter.split_documents(docs)

In [74]:
embedding = SentenceTransformer("Alibaba-NLP/gte-large-en-v1.5", trust_remote_code=True)

In [51]:
document_texts = [doc.page_content for doc in document_chunks]
document_embeddings = embedding.encode(document_texts)

In [65]:
vector_db = Chroma.from_documents(
    documents=document_chunks,
    embedding_function=embedding.encode
)

TypeError: langchain_community.vectorstores.chroma.Chroma() got multiple values for keyword argument 'embedding_function'

In [66]:
vector_db = Chroma(
    embedding_function=embedding.encode  # Pass the embedding function during initialization
)

/var/folders/l6/wl5zltz511d7fhrc7p1xmt6c0000gn/T/ipykernel_25040/2214342378.py:1: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vector_db = Chroma(


In [67]:
vector_db.add_documents(document_chunks)

AttributeError: 'function' object has no attribute 'embed_documents'

In [70]:
embedding_function = SentenceTransformerEmbeddings(embedding)

/var/folders/l6/wl5zltz511d7fhrc7p1xmt6c0000gn/T/ipykernel_25040/2730910636.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(embedding)


TypeError: HuggingFaceEmbeddings.__init__() takes 1 positional argument but 2 were given

In [72]:
from langchain.embeddings import HuggingFaceEmbeddings

In [75]:
embedding_function = HuggingFaceEmbeddings(model_name="dunzhang/stella_en_1.5B_v5")

RuntimeError: MPS backend out of memory (MPS allocated: 9.04 GB, other allocations: 656.00 KB, max allowed: 9.07 GB). Tried to allocate 52.50 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [76]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
embedding_function = HuggingFaceEmbeddings(model_name=model_name)

ValueError: Loading Alibaba-NLP/gte-large-en-v1.5 requires you to execute the configuration file in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

In [77]:
embedding_function = HuggingFaceEmbeddings(model_name=model_name, trust_remote_code=True)


ValidationError: 1 validation error for HuggingFaceEmbeddings
trust_remote_code
  Extra inputs are not permitted [type=extra_forbidden, input_value=True, input_type=bool]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

In [80]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

# Initialize HuggingFaceEmbeddings with the loaded model and tokenizer
embedding_function = HuggingFaceEmbeddings(model=model, tokenizer=tokenizer)

ValidationError: 2 validation errors for HuggingFaceEmbeddings
model
  Extra inputs are not permitted [type=extra_forbidden, input_value=NewModel(
  (embeddings):...lse)
      )
    )
  )
), input_type=NewModel]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
tokenizer
  Extra inputs are not permitted [type=extra_forbidden, input_value=BertTokenizerFast(name_or...alse, special=True),
}
), input_type=BertTokenizerFast]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

In [79]:
from transformers import AutoModel, AutoTokenizer

In [81]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"
model = AutoModel.from_pretrained(model_name, trust_remote_code=True)

# Initialize HuggingFaceEmbeddings with the model
embedding_function = HuggingFaceEmbeddings(model=model)

ValidationError: 1 validation error for HuggingFaceEmbeddings
model
  Extra inputs are not permitted [type=extra_forbidden, input_value=NewModel(
  (embeddings):...lse)
      )
    )
  )
), input_type=NewModel]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden

In [82]:
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_core.documents.base import Document

In [83]:
model_name = "Alibaba-NLP/gte-large-en-v1.5"

# Initialize HuggingFaceEmbeddings with the model name (not the model object)
embedding_function = HuggingFaceEmbeddings(model_name=model_name, model_kwargs={"trust_remote_code": True})


In [84]:
vector_db = Chroma.from_documents(
    documents=document_chunks,  # Pass the Document objects
    embedding_function=embedding_function  # Pass the correct HuggingFaceEmbeddings
)

TypeError: langchain_community.vectorstores.chroma.Chroma() got multiple values for keyword argument 'embedding_function'

In [85]:
vector_db = Chroma.from_documents(
    documents=document_chunks,  # Pass the Document objects
    embedding=embedding_function  # Correct argument name: `embedding` instead of `embedding_function`
)

In [86]:
# Retrieve

def _get_context_retriever_chain(vector_db, llm):
    retriever = vector_db.as_retriever()
    prompt = ChatPromptTemplate.from_messages([
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
        ("user", "Given the above conversation, generate a search query to look up in order to get inforamtion relevant to the conversation, focusing on the most recent messages."),
    ])
    retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

    return retriever_chain

In [87]:
def get_conversational_rag_chain(llm):
    retriever_chain = _get_context_retriever_chain(vector_db, llm)

    prompt = ChatPromptTemplate.from_messages([
        ("system",
        """You are a helpful assistant. You will have to answer to user's queries.
        You will have some context to help with your answers, but now always would be completely related or helpful.
        You can also use your knowledge to assist answering the user's queries.\n
        {context}"""),
        MessagesPlaceholder(variable_name="messages"),
        ("user", "{input}"),
    ])
    stuff_documents_chain = create_stuff_documents_chain(llm, prompt)

    return create_retrieval_chain(retriever_chain, stuff_documents_chain)

In [91]:
# Augmented Generation

llm_stream_gemini = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",  # Here you could use "o1-preview" or "o1-mini" if you already have access to them
    #temperature=0.3,
    #streaming=True,
    google_api_key=os.getenv('google_api_key')
)

llm_stream_anthropic = ChatAnthropic(
    model="claude-3-5-sonnet-20240620",
    temperature=0.3,
    streaming=True,
)

llm_stream = llm_stream_gemini # Select between OpenAI and Anthropic models for the response

messages = [
    {"role": "user", "content": "Hi"},
    {"role": "assistant", "content": "Hi there! How can I assist you today?"},
    {"role": "user", "content": "What is the latest version of Streamlit?"},
]
messages = [HumanMessage(content=m["content"]) if m["role"] == "user" else AIMessage(content=m["content"]) for m in messages]

conversation_rag_chain = get_conversational_rag_chain(llm_stream)
response_message = "*(RAG Response)*\n"
for chunk in conversation_rag_chain.pick("answer").stream({"messages": messages[:-1], "input": messages[-1].content}):
    response_message += chunk
    print(chunk, end="", flush=True)

messages.append({"role": "assistant", "content": response_message})

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


According to the provided release notes, the latest version of Streamlit is 1.41.0, released on December 10, 2024.
